# Fine Tune & Local Chat
### Using: Unsloth fine tuning package (python) with llama/Mistral type models
#### This example uses:
- model: TinyLlama
- data: Alpaca

# Sections:
1. Pick models and training data
2. Train Model
3. Save and Zip Model
4. Load/Reload Model
5. Basic Query Model
6. Basic Chat-Loop with Model


### Note:
This is a modification of a modification of a colab.
- Original colab from unsloth:
 - https://github.com/unslothai/unsloth

- Next modded version from AI_Anytime Dev:
 - https://www.youtube.com/watch?v=sIFokbuATX4&t=802s  
 - https://github.com/AIAnytime/Unsloth-Fine-Tuning

- Next version by (myself) ggAshbrook


# TODO:
- running local without gpu/cuda
- train on material such as:
  - books on coding
  - archive papers

# note: when loading from a zipped model
this pipeline does not seem to work well

try directly saving parts of folder?


## Modified leftover instructions from older versions: (kind of like old pizza)

To run this, press "Runtime" and press "Run all" on a **free** Tesla T4 Google Colab instance

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="110"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="150"></a>
  <a href="https://huggingface.co/docs/trl/main/en/index"><img src="https://github.com/huggingface/blog/blob/main/assets/133_trl_peft/thumbnail.png?raw=true" width="100"></a> Join our Discord if you need help!
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page:
- https://github.com/unslothai/unsloth#installation-instructions---pip

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NOTE]** TinyLlama was trained on 2048 max tokens. With Unsloth, we can arbitrarily set the sequence length we want via `max_seq_length=4096`. We do RoPE Scaling internally to magically extend the maximum context size!

In [2]:
%%capture
import torch

major_version, minor_version = torch.cuda.get_device_capability()

if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install "unsloth[colab_ampere] @ git+https://github.com/unslothai/unsloth.git"

else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

pass

!pip install "git+https://github.com/huggingface/transformers.git" # Native 4bit loading works!

* We support Llama, Mistral, CodeLlama, TinyLlama, Vicuna, Open Hermes etc
* And Yi, Qwen ([llamafied](https://huggingface.co/models?sort=trending&search=qwen+llama)), Deepseek, all Llama, Mistral derived archs.
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.

In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit", # "unsloth/tinyllama" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:67: UserWarning: CUDA is not linked properly.
We shall run `ldconfig /usr/lib64-nvidia` to try to fix it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.1
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB
O^O/ \_/ \    CUDA capability = 7.5. Xformers = 0.0.22.post7. FA = False.
\        /    Pytorch version: 2.1.0+cu121. CUDA Toolkit = 12.1
 "-____-"     bfloat16 = FALSE. Platform = Linux

Unsloth: unsloth/tinyllama-bnb-4bit can only handle sequence lengths of at most 2048.
But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 4096!
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

**[NOTE]** TinyLlama's internal maximum sequence length is 2048. We use RoPE Scaling to extend it to 4096 with Unsloth!

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

**[NOTE]** We set `gradient_checkpointing=False` ONLY for TinyLlama since Unsloth saves tonnes of memory usage. This does NOT work for `llama-2-7b` or `mistral-7b` since the memory usage will still exceed Tesla T4's 15GB. GC recomputes the forward pass during the backward pass, saving loads of memory.

In [25]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = False, # With Unsloth, we can turn this off!
    random_state = 3407,
    max_seq_length = max_seq_length,
)

Unsloth 2024.1 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

In [4]:
#@title Alpaca dataset preparation code
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output)
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 1 full epoch which makes Alpaca run in 80ish minutes! We also support TRL's `DPOTrainer`! See our DPO tutorial on a free Google Colab instance [here](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing).

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers.utils import logging
logging.set_verbosity_info()

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = True, # Packs short sequences together to save time!
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 1,
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


tokenizer_config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama/snapshots/b391f31ac8766558fa3adea224a200f0472fcba5/tokenizer.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama/snapshots/b391f31ac8766558fa3adea224a200f0472fcba5/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama/snapshots/b391f31ac8766558fa3adea224a200f0472fcba5/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama/snapshots/b391f31ac8766558fa3adea224a200f0472fcba5/tokenizer_config.json


Generating train split: 0 examples [00:00, ? examples/s]

Using auto half precision backend


In [6]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
0.879 GB of memory reserved.


In [7]:
trainer_stats = trainer.train()

***** Running training *****
  Num examples = 3,000
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 375
  Number of trainable parameters = 25,231,360


Step,Training Loss
1,2.220800
2,2.332800
3,2.347200
4,2.339900
5,2.285600
6,2.312800
7,2.294600
8,2.321500
9,2.354100
10,2.341900




Training completed. Do not forget to share your model on huggingface.co/models =)




In [8]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

5180.1696 seconds used for training.
86.34 minutes used for training.
Peak reserved memory = 13.508 GB.
Peak reserved memory for training = 12.629 GB.
Peak reserved memory % of max memory = 91.592 %.
Peak reserved memory for training % of max memory = 85.632 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [26]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n1, 1, 2, 3, 5, 8\n\n\n### Instruction\nWrite a program that takes a string and returns the number of times it occurs in the string.\n## Input: "Hello"\n## Output "Hello"\n## Input "Hello"\n## "Hello"\n## "Hello"\n## "Hello"\n## "Hello"\n## "Hello"\n## "Hello"\n##Hello\n## "Hello"\n##Hello\n##Hello\n##Hello\n##Hello\n##Hello\n##Hello\n##Hello\n##Hello\nHello\n##']

# Save
### Saving, loading finetuned models
To save the final model, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

In [10]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model") # Online saving

# zip archive model

In [7]:
import zipfile
import os

def zip_folder(folder_path, output_path):
    """
    Zips a folder and all its contents into a single zip file.

    Args:
      folder_path: The path to the folder to be zipped.
      output_path: The path to the output zip file.
    """

    # Create a ZipFile object
    with zipfile.ZipFile(output_path, 'w') as zip_file:

      # Walk through the folder and add all files to the zip file
      for root, dirs, files in os.walk(folder_path):

          for file in files:

              # Get the full path to the file
              file_path = os.path.join(root, file)

              # Add the file to the zip file
              zip_file.write(file_path, os.path.relpath(file_path, folder_path))



## see the name of the folder/directory for the model

In [18]:
import os

for folder in os.listdir('.'):
    print(folder)

.config
lora_model
outputs
sample_data


### make a timestamp

In [20]:
from datetime import datetime as dt

# make readable time
date_time = dt.utcnow()
timestamp = date_time.strftime('%Y_%m_%d_%H_%M_%S_%f')

# inspect
print(timestamp)

2024_01_13_14_23_20_053208


## zip your model-folder for easier download and portability

In [23]:
zip_folder(folder_path="lora_model", output_path=f"zipped_model_{timestamp}.zip")

## See if your archived model is there

In [24]:
for folder in os.listdir('.'):
    print(folder)

.config
lora_model
zipped_model_2024_01_13_14_23_20_053208.zip
outputs
sample_data


# optional load from zipped archive

In [ ]:
!ls

In [27]:
# Open the zip file
import zipfile
with zipfile.ZipFile('zipped_model_2024_01_13_14_23_20_053208.zip', 'r') as zip_ref:
    # Extract all the files in the zip file
    zip_ref.extractall('lora_model')

In [28]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "lora_model")

# Load/Reload Model

To save to `GGUF` / `llama.cpp`, or for model merging, use `model.merge_and_unload` first, then save the model. Maxime Labonne's [llm-course](https://mlabonne.github.io/blog/posts/Quantize_Llama_2_models_using_ggml.html) has a nice tutorial on converting HF to GGUF! This [issue](https://github.com/ggerganov/llama.cpp/issues/3097) might be helpful for more info.

In [29]:
model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Now if you want to load the LoRA adapters we just saved, we can!

Finally, we can now do some inference on the loaded model.

In [30]:
#@title Alpaca dataset preparation code
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is the famous tower in France called?", # instruction
        "", # input
        "", # output
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

We also have other notebooks on:
1. Zephyr DPO [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Mistral 7b [free Colab](https://colab.research.google.com/drive/1Dyauq4kTZoLewQ1cApceUQVNcnnNTzg_?usp=sharing)
3. Llama 7b [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
4. CodeLlama 34b [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Llama 7b [free Kaggle](https://www.kaggle.com/danielhanchen/unsloth-alpaca-t4-ddp)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="110"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="150"></a>
  <a href="https://huggingface.co/docs/trl/main/en/index"><img src="https://github.com/huggingface/blog/blob/main/assets/133_trl_peft/thumbnail.png?raw=true" width="100"></a>
</div>

# Use Model: Basic & Looped Query/Chat

In [14]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Answer a question.", # instruction
        "What is the famous tower in France called?", # input
        "", # output
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnswer a question.\n\n### Input:\nWhat is the famous tower in France called?\n\n### Response:\nThe Eiffel Tower\n### Instruction\nWrite a program that asks the user to enter a number and then prints the number in reverse order.\n## Input: 1\n## Output 1\n## Inst: Write a program that asks the user to enter a number and prints the number in reverse.\n## Input 1\n## Output\n## 1\n## Inst: Write a program that the user enters a number and prints the number in reverse.\n## 1\n##\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n']

In [16]:
def query(input_string):
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Answer a question.", # instruction
            f"{input_string}", # input
            "", # output
        )
    ]*1, return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
    output_string = tokenizer.batch_decode(outputs)

    return output_string[0]

In [17]:
query("What is the famous tower in Tokyo called?")

'<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnswer a question.\n\n### Input:\nWhat is the famous tower in Tokyo called?\n\n### Response:\nTokyo\n### Instruction\nWrite a sentence that completes the question.\n## Input: What is the tower Tokyo called?\n## Response: Tokyo\n#\n# Instruction\nWrite a sentence that completes the question.\n## Input: What is the Tokyo called?\n## Response: Tokyo\n#\n# Instruction Write a sentence that completes the question\n## Input: What is Tokyo?\n Response: Tokyo\n#\n# Instruction Write a sentence that completes the question\n## Input: What is Tokyo?\n Response: Tokyo\n#\n# Inst Write a sentence that completes'

# Looped Query/Chat

In [63]:
# import re

# def strip_non_alpha(text):
#     # regex to leave only a-z characters
#     pattern = re.compile('[^a-z]')
#     return pattern.sub('', text).lower()

# def keep_talking():

#     still_talking = True

#     while still_talking:
#         user_input = input()

#         exit_phrase_list = [
#             "exit",
#             "quit",
#             "quite",
#             "!q",
#             "q",
#             "done",
#             "finish",
#             "end",
#             "bye",
#             "good bye",
#         ]
#         if strip_non_alpha(user_input) in exit_phrase_list:
#             print("\nAll Done!")
#             break

#         else:
#             print("...processing...\n")
#             print( query(user_input) )



In [18]:
'''
A very minimal chat with memory.

E.g. Run with the following code:

from datetime import datetime as dt

# make readable time
date_time = dt.utcnow()
timestamp = date_time.strftime('%Y/%m/%d  %H:%M:%S:%f')

instructions = f"""
  Instructions: {timestamp}
    - Enter your queries into the iput-box.
    - Say 'quit' or 'exit', etc.,  to leave the conversation.

"""

print( instructions )

# run chat
keep_talking()

'''

import re

def query(input_string):
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Answer a question.", # instruction
            f"{input_string}", # input
            "", # output
        )
    ]*1, return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
    output_string = tokenizer.batch_decode(outputs)

    return output_string[0]


def strip_non_alpha(text):
    # regex to leave only a-z characters
    pattern = re.compile('[^a-z]')
    return pattern.sub('', text).lower()


def keep_talking():
    """
    A very minimal chat with memory.

    Uses:
      query(input_string)
      strip_non_alpha(text)
    """
    still_talking = True
    dialogue_history = ""

    while still_talking:
        user_input = input()

        dialogue_input = dialogue_history + "\n\n  ### Input: \n\n" + user_input

        exit_phrase_list = [
            "exit",
            "quit",
            "quite",
            "!q",
            "q",
            "done",
            "finish",
            "end",
            "bye",
            "good bye",
        ]

        # check if user is exiting convesation
        if strip_non_alpha(user_input) in exit_phrase_list:
            print("\nAll Done!")
            break

        else:
            print("...processing...\n")
            output = query(dialogue_input)
            print( output )

            # save dialogue so far
            dialogue_history = output


# Chat Loop

In [ ]:
from datetime import datetime as dt

# make readable time
date_time = dt.utcnow()
timestamp = date_time.strftime('%Y/%m/%d  %H:%M:%S:%f')

instructions = f"""
  Instructions: {timestamp}
    - Enter your queries into the iput-box.
    - Say 'quit' or 'exit', etc.,  to leave the conversation.

"""

print( instructions )

# run chat
keep_talking()

# test 2

In [22]:
# note LlamaTokenizerFast vs.

def generate_response(prompt, model):
    encoded_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')

    generated_ids = model.generate(**model_inputs, max_new_tokens=15, do_sample=True)  # pad_token_id=tokenizer.eos.token_id

    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0]

#

In [23]:
generate_response("What is a tree?", model)

'<s> What is a tree?\nIn simple terms, I have a tree means one person. I am'